In [5]:
import json
import requests
import random
from IPython.display import display, Audio, clear_output

In [9]:
def getURL(term):
    keyVals = {'term':term, 'media':'music'}
    link = requests.get('https://itunes.apple.com/search', params=keyVals)
    return link.json()

def write(data):
    with open('json_data.json','w') as outfile:
        json.dump(data, outfile)

def getListOfSongs():
    songlst = []
    while True:
        userInp = input('Enter the name of a band or artist (or "done" to exit): ')
        if userInp == 'done':
            return songlst
        else:
            data = getURL(userInp)
            songlst += data['results']
            print(len(songlst),'song total \n')

def playSong(data):
    audio_url = data['previewUrl']
    Audio(url=audio_url)
    display(Audio(audio_url, autoplay=True))

def getName(data):
    name = data['trackName']
    blankName = ''
    for char in name:
        if char.isalpha():
            blankName += '_'
        elif char.isnumeric():
            blankName += '_'
        else:
            blankName += char
    return blankName

def guessSong(data, guess):
    name = data['trackName']
    
    if guess.upper() == name.upper():
        print('You got it!')
        return
    elif guess.lower() == 'skip':
        print("The song was '{}'".format(name))
        return
    elif guess.lower() == 'exit':
        print("The song was '{}'".format(name))
        return 'exit'
    else:
        print("It's not '{}'".format(guess))
        new_guess = askForHint(data, guess)

def askForHint(data, guess):
    new_guess = guess
    song_name = data['trackName']
    need_hint = input('Need a hint? Type "y" for yes or "n" for no: ')

    if need_hint != 'y' and need_hint != 'n':
        print('Please type a valid input. ')
    elif need_hint == 'y':
        which_hint = input('Type "h" for hangman or "p" for album picture: ')
        if which_hint == 'p':
            picture(data)
        if which_hint == 'h':
            outcome = hangman(data)
            if outcome[0] == 'Unfinished':
                print('You ran out of tries.')
            else:
                print('Congrats! You got it in {} tries.'.format(outcome[1]))
            new_guess = outcome[2]
    elif need_hint == 'n':
        pass
    return new_guess

def picture(data):
    return display(Image(data['artworkUrl100']))
        
def hangman(data):
    """
    song[string]:song to be guessed
    progress[string]:song with "_" for unguessed letters;
        special symbols are already pre-filled
    """
    song = data['trackName']
    progress = getName(data)
    

    guessed = False
    guessedLetters = []
    tries = 6
    
    print("Welcome to Song Hangman!")
    print(progress)
    print("\n")
    while not guessed and tries != 0:
        guess = input("Guess a letter: ").upper()
        if guess.isalpha() and len(guess) == 1 and guess in song.upper():
            if guess in guessedLetters:
                print("You have already guessed this letter.")
            else:
                guessedLetters.append(guess)
                progressLst = list(progress)
                charIndices = [i for i,char in enumerate(song.upper()) if guess == char]
                for i in charIndices:
                    progressLst[i] = song[i]
                progress = "".join(progressLst)
                if "_" not in progress:
                    guessed = True
            
        elif guess.isalpha() and len(guess) == 1 and guess not in song.upper():
            if guess in guessedLetters:
                print("You have already guessed this letter.")
            else:
                tries-=1
                guessedLetters.append(guess)
                print("That letter is not in the song! Try again.")
        else:
            print("Please type in a valid guess.")
            
        print("You have ",tries," tries remaining.")
        print(progress)
        print("\n")
    
    if guessed:
        print("Great job! You guessed the song.")
    else:
        print("Better luck next time.")
    
def play():
    round_num = 1
    proceed = True
    songs = getListOfSongs()
    while proceed:
        print('ROUND {}'.format(round_num))
        song = random.choice(songs)
        playSong(song)
        guess = input("Guess the song (or 'skip' or 'exit'): ")
        if guess.lower() == 'exit':
            proceed = False
        guessSong(song, guess)
        
        
        
        
        round_num += 1
        
play()



Enter the name of a band or artist (or "done" to exit): maroon 5
50 song total 

Enter the name of a band or artist (or "done" to exit): done
ROUND 1


Guess the song (or 'skip' or 'exit'): map
It's not 'map'
Need a hint? Type "y" for yes or "n" for no: y
Type "h" for hangman or "p" for album picture: h
Welcome to Song Hangman!
____


Guess a letter: m
You have  6  tries remaining.
M___


Guess a letter: a
You have  6  tries remaining.
Ma__


Guess a letter: p
You have  6  tries remaining.
Map_


Guess a letter: s
You have  6  tries remaining.
Maps


Great job! You guessed the song.


TypeError: 'NoneType' object is not subscriptable